# Ephemeris generation
##### In this notebook, we demonstrate how to generate and analyze the ephemerides of a spacecraft. By extrapolating the spacecraft's trajectory along with its covariance, we obtain the ephemerides in a CCSDS OEM format. Additionally, we plot the orbital elements to visualize the results. You will learn how to configure the initial orbital state using TLE data, set up the propagation context, and create the necessary models to generate accurate ephemerides.

## Jupyter Notebook Help Guide
Welcome to your Jupyter Notebook! If you need assistance or more information about any function, method, or object, the contextual help window is a valuable tool.

***Important - To have access to contextual help, start by running the [Imports](#imports) section below***

![Run Notebook](https://portal.exotrail.space/images/products/notebooks/execute_workflow.jpg)


### How to Open the Contextual Help Window


1. **Use the Help Menu**:
   - Navigate to the top menu bar and click on `Help`. From the dropdown menu, select `Show Contextual Help`.
2. **Keyboard Shortcut**:
   - Press `Shift + Tab` while your cursor is over the code you want more information on. For a more detailed view, press `Shift + Tab` twice.
3. **Using the Inspector**:
   - Type `?` followed by the function or object name and run the cell. For example, `?print`.
   - To see the full documentation, use `??` instead, e.g., `??print`.


### Tips for Using the Contextual Help


- **Inline Help**: Single press `Shift + Tab` to get a brief pop-up of the docstring.
- **Expanded Help**: Double press `Shift + Tab` or click the expand icon in the pop-up to open the full documentation in the help pane.
- **Persistent Help Pane**: Use the Help menu or `Shift + Tab` twice to dock the help pane on the right side of the screen, where it can stay open as you work.
This feature can help you understand function parameters, return types, and see example usages directly in your notebook. Happy coding!

## Imports

In [ ]:
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fds.models.orbit_extrapolation.requests import OemRequest, EphemeridesRequest
from fds.models.orbit_extrapolation.use_case import OrbitExtrapolation
from fds.models.orbital_state import PropagationContext, CovarianceMatrix, OrbitalState
from fds.models.spacecraft import Battery, SolarArray, ThrusterElectrical, SpacecraftBox
from fds.models.two_line_element import TwoLineElement
from fds.utils.frames import Frame
from fds.utils.plotting import add_grid

*Note*: If you are using this notebook locally, use the following methods to configure your credentials:

```python
from fds.config import set_client_id, set_client_secret
set_client_id('CLIENT_ID')
set_client_secret('CLIENT_SECRET')
```

## Create models

### Create Orbital State
`OrbitalState` is the object used in spacetower to group information on the initial orbit, the propagation context and the spacecraft definition.  

#### TLE

The TLE provides the orbital parameters needed for accurate satellite tracking and prediction.

In [ ]:
tle = TwoLineElement(
    "1 25544U 98067A   24142.35003124  .00022843  00000-0  38371-3 0  9995",
    "2 25544  51.6390  88.3709 0003333 191.4959 306.2513 15.51667899454382"
    )

print(f"TLE date (UTC): {tle.date}")

#### Propagation Context

The `PropagationContext` object encapsulates the settings necessary to accurately propagate the satellite's orbit by accounting for gravitational and non-gravitational perturbations. Adjusting these parameters allows for fine-tuning the balance between computational efficiency and the accuracy of the simulation.

In [ ]:
propagation_context = PropagationContext(
    model_perturbations=[
        PropagationContext.Perturbation.DRAG,
        PropagationContext.Perturbation.EARTH_POTENTIAL,
        PropagationContext.Perturbation.SRP,
        PropagationContext.Perturbation.THIRD_BODY,
    ],
    model_solar_flux=150,  # SFU
    model_earth_potential_deg=30,
    model_earth_potential_ord=30,
    model_atmosphere_kind=PropagationContext.AtmosphereModel.HARRIS_PRIESTER,
    integrator_kind=PropagationContext.IntegratorKind.DORMAND_PRINCE_853,
    integrator_min_step=0.01,  # s
    integrator_max_step=100,  # s
)

#### Spacecraft

Here we are using the `SpacecraftBox` class to define our spacecraft.  
To define a simpler model, try using the `SpacecraftSphere` class where battery, solar-arrays and thrusters do not need to be defined.

In [ ]:
battery = Battery(
    depth_of_discharge=0.3,  # 0<x<1
    nominal_capacity=560,  # W
    minimum_charge_for_firing=0.9,  # 0<x<1
    initial_charge=1,  # 0<x<1
)

solar_array = SolarArray(
    kind="DEPLOYABLE_FIXED",
    initialisation_kind=SolarArray.InitialisationKind.MAXIMUM_POWER,
    efficiency=.293,  # 0<x<1
    normal_in_satellite_frame=(0, 0, -1),  # Unit vector
    maximum_power=300,  # W
    surface=0.75,  # m^2
)

electrical_thruster = ThrusterElectrical(
    isp=950,  # s
    thrust=0.005,  # N
    axis_in_satellite_frame=(-1, 0, 0),  # Unit vector
    propellant_mass=4,  # kg
    wet_mass=11,  # kg
    warm_up_duration=240,  # s
    maximum_thrust_duration=1200,  # s
    impulse=37265.27,  # Ns
    power=300,  # W
    stand_by_power=1.1,  # W
    warm_up_power=50,  # W
)

spacecraft = SpacecraftBox(
    battery=battery,
    thruster=electrical_thruster,
    solar_array=solar_array,
    platform_mass=112,  # kg
    drag_coefficient=2.2,
    length_x=.5,  # m
    length_y=.5,  # m
    length_z=.5,  # m
    max_angular_velocity=2,  # deg/s
    max_angular_acceleration=.5,  # deg/s^2
)

#### Covariance

The initial state covariance matrix represents the uncertainty in the spacecraft's initial position and velocity. It quantifies how precise the initial estimates are, with larger values indicating greater uncertainty

In [ ]:
covariance = CovarianceMatrix.from_diagonal(
    diagonal=(100, 100, 100, 0.1, 0.1, 0.1),
    frame=Frame.TNW
)

#### Orbital State

Initiate the `OrbitalState` object with the TLE, the propagation context, the spacecraft, and the covariance matrix.

In [ ]:
orbital_state = OrbitalState.from_tle(
    tle=tle,
    covariance_matrix=covariance,
    propagation_context=propagation_context,
    spacecraft=spacecraft,
)

### Create Ephermeris Generation Configuration
In this section, we set up the parameters for generating ephemeris data. We create an Orbit Data Message request and define the Ephemerides Request parameters to determine the type of ephemeris data (Keplerian) and its generation interval.

In [ ]:
# Orbit Data Message request
oem_request = OemRequest(
    creator="User",
    object_id="25544",
    object_name="ISS",
    frame=Frame.EME2000,
    write_acceleration=False,
    write_covariance=True,
    ephemerides_step=60,
)

ephemerides_request = EphemeridesRequest(
    ephemeris_types=[EphemeridesRequest.EphemerisType.KEPLERIAN],
    step=200,
)

## Build & Run Use Case

In [ ]:
oe = OrbitExtrapolation.with_target_date(
    target_date=orbital_state.date + datetime.timedelta(seconds=orbital_state.mean_orbit.keplerian_period * 3),
    initial_orbital_state=orbital_state,
    orbit_data_message_request=oem_request,
    ephemerides_request=ephemerides_request,
    extrapolate_covariance=True
)
print("Propagation of approximately 3 orbits since the TLE date")
print(f"Start date: {oe.initial_date}")
print(f"End date: {oe.final_date}")
print(f"Duration: {format(oe.duration / 3600., '.3f')} hours")

# Run the orbit extrapolation
oe.run()

## Results/Post-Processing

### Plot Results

#### Osculating Keplerian Elements

In [ ]:
ephemerides = oe.result.export_keplerian_ephemeris()
eph_df = pd.DataFrame(ephemerides)

# plot
fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=True)

axs[0].plot(eph_df['date'], eph_df['semi_major_axis'], color='tab:blue')
axs[0].set_ylabel('Semi-major axis [km]')

axs[1].plot(eph_df['date'], eph_df['eccentricity'], color='tab:orange')
axs[1].set_ylabel('Eccentricity')

axs[2].plot(eph_df['date'], eph_df['inclination'], color='tab:green')
axs[2].set_ylabel('Inclination [deg]')

for ax in axs:
    add_grid(ax)

fig.autofmt_xdate()
fig.align_ylabels()
fig.suptitle('Osculating Keplerian elements')
fig.tight_layout()
plt.show()

### Print CCSDS Orbit Ephemeris Message
Position-velocity ephemeris in inertial frame

In [ ]:
print(oe.result.orbit_data_message)